In [2]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np 
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

In [3]:
path='./'
train = pd.read_csv(path+'input/pfm_train.csv')
test = pd.read_csv(path+'input/pfm_test.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 31 columns):
Age                         1100 non-null int64
Attrition                   1100 non-null int64
BusinessTravel              1100 non-null object
Department                  1100 non-null object
DistanceFromHome            1100 non-null int64
Education                   1100 non-null int64
EducationField              1100 non-null object
EmployeeNumber              1100 non-null int64
EnvironmentSatisfaction     1100 non-null int64
Gender                      1100 non-null object
JobInvolvement              1100 non-null int64
JobLevel                    1100 non-null int64
JobRole                     1100 non-null object
JobSatisfaction             1100 non-null int64
MaritalStatus               1100 non-null object
MonthlyIncome               1100 non-null int64
NumCompaniesWorked          1100 non-null int64
Over18                      1100 non-null object
OverTime              

In [4]:
train.describe()

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,...,1100.000000,1100.0,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000,1100.000000
mean,36.999091,0.161818,9.427273,2.922727,1028.157273,2.725455,2.730909,2.054545,2.732727,6483.620909,...,2.696364,80.0,0.788182,11.221818,2.807273,2.746364,7.011818,4.207273,2.226364,4.123636
std,9.037230,0.368451,8.196694,1.022242,598.915204,1.098053,0.706366,1.107805,1.109731,4715.293419,...,1.095356,0.0,0.843347,7.825548,1.291514,0.701121,6.223093,3.618115,3.313830,3.597996
min,18.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1009.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,0.000000,2.000000,2.000000,504.250000,2.000000,2.000000,1.000000,2.000000,2924.500000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,0.000000,7.000000,3.000000,1026.500000,3.000000,3.000000,2.000000,3.000000,4857.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,0.000000,15.000000,4.000000,1556.500000,4.000000,3.000000,3.000000,4.000000,8354.500000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1.000000,29.000000,5.000000,2065.000000,4.000000,4.000000,5.000000,4.000000,19999.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,37.000000,18.000000,15.000000,17.000000


In [5]:
test.describe()

,Age,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.0,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,36.471429,8.391429,2.868571,1023.285714,2.714286,2.734286,2.068571,2.725714,6479.491429,2.691429,...,2.745714,80.0,0.817143,11.202857,2.782857,2.808571,6.782857,4.260000,1.951429,4.017143
std,9.373378,7.685318,1.029583,612.566819,1.067129,0.726669,1.089615,1.083437,4633.609813,2.456892,...,1.041226,0.0,0.886539,7.470399,1.295238,0.722488,5.489113,3.622336,2.752532,3.383720
min,18.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1051.000000,0.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,2.000000,2.000000,463.250000,2.000000,2.000000,1.000000,2.000000,2888.750000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,35.000000,7.000000,3.000000,1011.000000,3.000000,3.000000,2.000000,3.000000,5104.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,42.000000,11.000000,4.000000,1584.500000,4.000000,3.000000,3.000000,4.000000,8260.250000,4.000000,...,4.000000,80.0,1.000000,15.750000,3.000000,3.000000,9.750000,7.000000,2.000000,7.000000
max,60.000000,29.000000,5.000000,2068.000000,4.000000,4.000000,5.000000,4.000000,19973.000000,9.000000,...,4.000000,80.0,3.000000,37.000000,6.000000,4.000000,29.000000,16.000000,15.000000,14.000000


In [5]:
train.corr()

,Age,Attrition,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,-0.175393,0.007081,0.198558,-0.010953,0.011803,0.066528,0.513882,-0.003744,0.500163,...,0.063489,NaN,-0.002413,0.682879,-0.051702,-0.001042,0.328651,0.231842,0.230587,0.212540
Attrition,-0.175393,1.000000,0.088563,-0.046494,-0.045168,-0.097003,-0.122722,-0.168775,-0.125568,-0.155521,...,-0.051749,NaN,-0.138498,-0.187922,-0.043395,-0.048794,-0.143697,-0.163059,-0.071760,-0.158558
DistanceFromHome,0.007081,0.088563,1.000000,0.011437,0.029930,-0.010308,0.012333,0.016470,-0.009641,-0.017757,...,0.018112,NaN,0.050356,0.001287,-0.041208,-0.050950,0.000044,0.019317,-0.002760,0.008852
Education,0.198558,-0.046494,0.011437,1.000000,0.055979,-0.032698,0.022843,0.084075,-0.010201,0.079834,...,-0.006346,NaN,0.035881,0.125672,-0.021629,0.003099,0.074522,0.064363,0.067754,0.071870
EmployeeNumber,-0.010953,-0.045168,0.029930,0.055979,1.000000,0.030428,-0.008589,-0.006121,-0.042443,-0.007147,...,-0.073687,NaN,0.049967,-0.009526,0.017175,0.000548,-0.011106,-0.005649,-0.019621,-0.005333
EnvironmentSatisfaction,0.011803,-0.097003,-0.010308,-0.032698,0.030428,1.000000,-0.028467,-0.015355,0.000212,-0.026410,...,0.033515,NaN,0.008874,-0.018532,-0.045686,0.026477,-0.012574,0.003572,0.008843,-0.020190
JobInvolvement,0.066528,-0.122722,0.012333,0.022843,-0.008589,-0.028467,1.000000,0.005983,-0.016382,0.006114,...,0.048363,NaN,0.029483,0.018380,-0.018001,-0.025862,-0.032189,0.001194,-0.031097,0.014176
JobLevel,0.513882,-0.168775,0.016470,0.084075,-0.006121,-0.015355,0.005983,1.000000,-0.005894,0.950776,...,0.042156,NaN,0.002638,0.784020,-0.034620,0.041258,0.544091,0.411481,0.395195,0.376119
JobSatisfaction,-0.003744,-0.125568,-0.009641,-0.010201,-0.042443,0.000212,-0.016382,-0.005894,1.000000,-0.009752,...,-0.033138,NaN,0.021123,-0.023343,0.002754,-0.042767,-0.013772,-0.011798,-0.009761,-0.041852
MonthlyIncome,0.500163,-0.155521,-0.017757,0.079834,-0.007147,-0.026410,0.006114,0.950776,-0.009752,1.000000,...,0.050121,NaN,-0.006320,0.772043,-0.039713,0.032247,0.520641,0.388163,0.386875,0.344237


In [8]:
test['Attrition']=-1
print(len(test.columns))

31


In [10]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).rese_index(drop=True)
print(data.head())

   Age  Attrition     BusinessTravel              Department  \
0   37          0      Travel_Rarely  Research & Development   
1   54          0  Travel_Frequently  Research & Development   
2   34          1  Travel_Frequently  Research & Development   
3   39          0      Travel_Rarely  Research & Development   
4   28          1  Travel_Frequently  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeNumber  \
0                 1          4  Life Sciences              77   
1                 1          4  Life Sciences            1245   
2                 7          3  Life Sciences             147   
3                 1          1  Life Sciences            1026   
4                 1          3        Medical            1111   

   EnvironmentSatisfaction  Gender          ...           \
0                        1    Male          ...            
1                        4  Female          ...            
2                        1    Male          

In [11]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [17]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['Attrition']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
#     data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

In [18]:
data['Department'].head()

0    1
1    1
2    1
3    1
4    1
Name: Department, dtype: int32

In [19]:
data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,2,1,1,4,1,77,1,1,...,3,80,1,7,2,4,7,5,0,7
1,54,0,1,1,1,4,1,1245,4,0,...,1,80,1,33,2,1,5,4,1,4
2,34,1,1,1,7,3,1,147,1,1,...,4,80,0,9,3,3,9,7,0,6
3,39,0,2,1,1,1,1,1026,4,0,...,3,80,1,21,3,3,21,6,11,8
4,28,1,1,1,1,3,3,1111,1,1,...,1,80,2,1,2,3,1,0,0,0


In [20]:
feats = [i for i in data.columns if i not in ['Attrition']]
feats

['Age',
 'BusinessTravel',
 'Department',
 'DistanceFromHome',
 'Education',
 'EducationField',
 'EmployeeNumber',
 'EnvironmentSatisfaction',
 'Gender',
 'JobInvolvement',
 'JobLevel',
 'JobRole',
 'JobSatisfaction',
 'MaritalStatus',
 'MonthlyIncome',
 'NumCompaniesWorked',
 'Over18',
 'OverTime',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StandardHours',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

In [21]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=70, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'F1',
    subsample=0.95, colsample_bytree=0.95, subsample_freq=1,
    learning_rate=0.002, random_state=2017
    )
train_x =data[data['Attrition']!=-1][feats]
train_y =data[data['Attrition']!=-1]['Attrition']
testx= data[data['Attrition']==-1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [25]:
#5折交叉验证
from sklearn.model_selection import KFold
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
data.loc[data['Attrition']==2,'Attrition']=0
train_x = data[data['Attrition']!=-1][feats]
train_y = data[data['Attrition']!=-1]['Attrition']
res=data[data['Attrition']==-1][['Attrition']]

test_x= data[data['Attrition']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5



[1]	training's auc: 0.878637	valid_1's auc: 0.716571
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.901984	valid_1's auc: 0.727302
[3]	training's auc: 0.915537	valid_1's auc: 0.760952
[4]	training's auc: 0.916644	valid_1's auc: 0.756571
[5]	training's auc: 0.926307	valid_1's auc: 0.754857
[6]	training's auc: 0.92684	valid_1's auc: 0.753333
[7]	training's auc: 0.928602	valid_1's auc: 0.751238
[8]	training's auc: 0.928894	valid_1's auc: 0.757841
[9]	training's auc: 0.931405	valid_1's auc: 0.757968
[10]	training's auc: 0.931601	valid_1's auc: 0.763048
[11]	training's auc: 0.932437	valid_1's auc: 0.765587
[12]	training's auc: 0.93374	valid_1's auc: 0.766476
[13]	training's auc: 0.935486	valid_1's auc: 0.764444
[14]	training's auc: 0.935793	valid_1's auc: 0.769524
[15]	training's auc: 0.93532	valid_1's auc: 0.769016
[16]	training's auc: 0.9353	valid_1's auc: 0.771937
[17]	training's auc: 0.934897	valid_1's auc: 0.771175
[18]	training's auc: 0.934424	val

[25]	training's auc: 0.926998	valid_1's auc: 0.742448
[26]	training's auc: 0.926868	valid_1's auc: 0.742618
[27]	training's auc: 0.926052	valid_1's auc: 0.744496
[28]	training's auc: 0.925653	valid_1's auc: 0.744496
[29]	training's auc: 0.926404	valid_1's auc: 0.742789
[30]	training's auc: 0.927026	valid_1's auc: 0.743642
[31]	training's auc: 0.927379	valid_1's auc: 0.741253
[32]	training's auc: 0.927574	valid_1's auc: 0.739717
[33]	training's auc: 0.92788	valid_1's auc: 0.741594
[34]	training's auc: 0.927444	valid_1's auc: 0.741765
[35]	training's auc: 0.927091	valid_1's auc: 0.741765
[36]	training's auc: 0.927175	valid_1's auc: 0.745349
[37]	training's auc: 0.927453	valid_1's auc: 0.745178
[38]	training's auc: 0.927249	valid_1's auc: 0.74313
[39]	training's auc: 0.927527	valid_1's auc: 0.741935
[40]	training's auc: 0.927778	valid_1's auc: 0.738863
[41]	training's auc: 0.927592	valid_1's auc: 0.738863
[42]	training's auc: 0.928539	valid_1's auc: 0.738351
[43]	training's auc: 0.93008	v

[113]	training's auc: 0.949989	valid_1's auc: 0.774306
[114]	training's auc: 0.949979	valid_1's auc: 0.774909
[115]	training's auc: 0.950179	valid_1's auc: 0.774004
[116]	training's auc: 0.950179	valid_1's auc: 0.7734
[117]	training's auc: 0.949989	valid_1's auc: 0.77189
[118]	training's auc: 0.950132	valid_1's auc: 0.77189
[119]	training's auc: 0.95016	valid_1's auc: 0.772947
[120]	training's auc: 0.949998	valid_1's auc: 0.772494
[121]	training's auc: 0.950074	valid_1's auc: 0.772343
[122]	training's auc: 0.950427	valid_1's auc: 0.77189
[123]	training's auc: 0.950437	valid_1's auc: 0.77189
[124]	training's auc: 0.950475	valid_1's auc: 0.771286
[125]	training's auc: 0.950532	valid_1's auc: 0.770833
[126]	training's auc: 0.950666	valid_1's auc: 0.7734
[127]	training's auc: 0.950637	valid_1's auc: 0.773551
[128]	training's auc: 0.950838	valid_1's auc: 0.773098
[129]	training's auc: 0.951105	valid_1's auc: 0.772494
[130]	training's auc: 0.95122	valid_1's auc: 0.773853
[131]	training's auc

[411]	training's auc: 0.97478	valid_1's auc: 0.77506
[412]	training's auc: 0.974799	valid_1's auc: 0.775211
[413]	training's auc: 0.974913	valid_1's auc: 0.775211
[414]	training's auc: 0.975056	valid_1's auc: 0.775211
[415]	training's auc: 0.975094	valid_1's auc: 0.774909
[416]	training's auc: 0.975152	valid_1's auc: 0.774758
[417]	training's auc: 0.975152	valid_1's auc: 0.775362
[418]	training's auc: 0.97519	valid_1's auc: 0.775211
[419]	training's auc: 0.975285	valid_1's auc: 0.775513
[420]	training's auc: 0.975371	valid_1's auc: 0.775211
[421]	training's auc: 0.9754	valid_1's auc: 0.775513
[422]	training's auc: 0.975457	valid_1's auc: 0.775815
[423]	training's auc: 0.975476	valid_1's auc: 0.775815
[424]	training's auc: 0.975524	valid_1's auc: 0.776117
[425]	training's auc: 0.975591	valid_1's auc: 0.77657
[426]	training's auc: 0.975686	valid_1's auc: 0.777023
[427]	training's auc: 0.975762	valid_1's auc: 0.776268
[428]	training's auc: 0.975762	valid_1's auc: 0.776872
[429]	training's

In [15]:
pd.DataFrame({'result':res['pred'].vulues}).to_csv(path+'output/lgb_proba.csv',index=False)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
model=GradientBoostingClassifier()
testX=test_x.values
trainY=train_y.values

trainX=train_x.values
model.fit(trainX,trainY)
predictY=model.predict_proba(testX)[:,1]

In [17]:
pd.DataFrame({'result':predictY}).to_csv(path+'output/gbdt_proba.csv',index=False)

In [26]:
res.loc[res['pred']>=0.5,'pred']=1
res.loc[res['pred']<0.5,'pred']=0
res.columns=['Attrition','result']
res['result']=res['result'].astype(int)
res[['result']].to_csv(path+'output/lgb_submission_5_fold.csv', index=False)